In [17]:
pip install scipy numpy pandas scikit-learn seaborn matplotlib pytdms h5py joblib streamlit tqdm


In [18]:
import os
import numpy as np
import pandas as pd
from scipy.io import loadmat
from nptdms import TdmsFile
import matplotlib.pyplot as plt
from tqdm import tqdm


In [22]:
vibration_path = r"C:/Users/MSI/digital Twin stimulator/VATM Dataset/vibration/"
acoustic_path = r"C:/Users/MSI/digital Twin stimulator/VATM Dataset/acoustic/"
current_temp_path = r"C:/Users/MSI/digital Twin stimulator/VATM Dataset/current,temp/"


In [23]:
def parse_filename(fname):
    name = fname.replace(".mat", "").replace(".tdms", "")
    parts = name.split("_")
    load = parts[0]                 # e.g. 4Nm
    fault = parts[1]                # e.g. BPFO
    severity = parts[2] if len(parts) > 2 else "None"
    return load, fault, severity


In [24]:
from scipy.fft import fft

def extract_features(signal):
    features = {}
    features['mean'] = np.mean(signal)
    features['std'] = np.std(signal)
    features['rms'] = np.sqrt(np.mean(signal**2))
    features['kurtosis'] = pd.Series(signal).kurt()
    features['skew'] = pd.Series(signal).skew()
    # FFT features
    fft_vals = np.abs(fft(signal))
    features['fft_peak'] = np.argmax(fft_vals)
    features['fft_mean'] = np.mean(fft_vals)
    return features


In [25]:
def extract_features(signal):
    signal = np.array(signal).astype(float).flatten()
    features = {}
    features['mean'] = np.mean(signal)
    features['std'] = np.std(signal)
    features['rms'] = np.sqrt(np.mean(signal**2))
    features['kurtosis'] = pd.Series(signal).kurt()
    features['skew'] = pd.Series(signal).skew()
    
    # Frequency features
    fft_vals = np.abs(fft(signal))
    features['fft_peak'] = np.argmax(fft_vals)
    features['fft_mean'] = np.mean(fft_vals)
    return features


In [26]:
vibration_records = []

for file in tqdm(os.listdir(vibration_path)):
    if file.endswith('.mat'):
        load, fault, severity = parse_filename(file)
        try:
            mat = loadmat(os.path.join(vibration_path, file))
            # Adjust key name according to your .mat structure
            key = [k for k in mat.keys() if not k.startswith('__')][0]
            data = mat[key]
            if isinstance(data, np.ndarray):
                # If multiple columns, process each direction and take average
                if data.ndim == 2:
                    combined = np.mean(data, axis=1)
                else:
                    combined = data.flatten()
                feats = extract_features(combined)
                feats.update({'File': file, 'Load': load, 'FaultType': fault, 'Severity': severity, 'Type': 'Vibration'})
                vibration_records.append(feats)
        except Exception as e:
            print(f"Error reading {file}: {e}")

vibration_df = pd.DataFrame(vibration_records)
print("Vibration data loaded:", vibration_df.shape)


  4%|███▋                                                                               | 2/45 [00:00<00:03, 12.26it/s]

Error reading 0Nm_BPFI_03.mat: ufunc 'add' did not contain a loop with signature matching types (dtype([('x_values', 'O'), ('y_values', 'O'), ('function_record', 'O')]), dtype([('x_values', 'O'), ('y_values', 'O'), ('function_record', 'O')])) -> None
Error reading 0Nm_BPFI_10.mat: ufunc 'add' did not contain a loop with signature matching types (dtype([('x_values', 'O'), ('y_values', 'O'), ('function_record', 'O')]), dtype([('x_values', 'O'), ('y_values', 'O'), ('function_record', 'O')])) -> None
Error reading 0Nm_BPFI_30.mat: ufunc 'add' did not contain a loop with signature matching types (dtype([('x_values', 'O'), ('y_values', 'O'), ('function_record', 'O')]), dtype([('x_values', 'O'), ('y_values', 'O'), ('function_record', 'O')])) -> None


 13%|███████████                                                                        | 6/45 [00:00<00:03, 12.09it/s]

Error reading 0Nm_BPFO_03.mat: ufunc 'add' did not contain a loop with signature matching types (dtype([('x_values', 'O'), ('y_values', 'O'), ('function_record', 'O')]), dtype([('x_values', 'O'), ('y_values', 'O'), ('function_record', 'O')])) -> None
Error reading 0Nm_BPFO_10.mat: ufunc 'add' did not contain a loop with signature matching types (dtype([('x_values', 'O'), ('y_values', 'O'), ('function_record', 'O')]), dtype([('x_values', 'O'), ('y_values', 'O'), ('function_record', 'O')])) -> None
Error reading 0Nm_BPFO_30.mat: ufunc 'add' did not contain a loop with signature matching types (dtype([('x_values', 'O'), ('y_values', 'O'), ('function_record', 'O')]), dtype([('x_values', 'O'), ('y_values', 'O'), ('function_record', 'O')])) -> None


 18%|██████████████▊                                                                    | 8/45 [00:00<00:04,  8.35it/s]

...

Error reading 4Nm_Normal.mat: ufunc 'add' did not contain a loop with signature matching types (dtype([('x_values', 'O'), ('y_values', 'O'), ('function_record', 'O')]), dtype([('x_values', 'O'), ('y_values', 'O'), ('function_record', 'O')])) -> None
Error reading 4Nm_Unbalance_0583mg.mat: ufunc 'add' did not contain a loop with signature matching types (dtype([('x_values', 'O'), ('y_values', 'O'), ('function_record', 'O')]), dtype([('x_values', 'O'), ('y_values', 'O'), ('function_record', 'O')])) -> None


 96%|██████████████████████████████████████████████████████████████████████████████▎   | 43/45 [00:06<00:00,  6.18it/s]

Error reading 4Nm_Unbalance_1169mg.mat: ufunc 'add' did not contain a loop with signature matching types (dtype([('x_values', 'O'), ('y_values', 'O'), ('function_record', 'O')]), dtype([('x_values', 'O'), ('y_values', 'O'), ('function_record', 'O')])) -> None
Error reading 4Nm_Unbalance_1751mg.mat: ufunc 'add' did not contain a loop with signature matching types (dtype([('x_values', 'O'), ('y_values', 'O'), ('function_record', 'O')]), dtype([('x_values', 'O'), ('y_values', 'O'), ('function_record', 'O')])) -> None


100%|██████████████████████████████████████████████████████████████████████████████████| 45/45 [00:06<00:00,  6.82it/s]

Error reading 4Nm_Unbalance_2239mg.mat: ufunc 'add' did not contain a loop with signature matching types (dtype([('x_values', 'O'), ('y_values', 'O'), ('function_record', 'O')]), dtype([('x_values', 'O'), ('y_values', 'O'), ('function_record', 'O')])) -> None
Error reading 4Nm_Unbalance_3318mg.mat: ufunc 'add' did not contain a loop with signature matching types (dtype([('x_values', 'O'), ('y_values', 'O'), ('function_record', 'O')]), dtype([('x_values', 'O'), ('y_values', 'O'), ('function_record', 'O')])) -> None
Vibration data loaded: (0, 0)


In [27]:
acoustic_records = []

for file in tqdm(os.listdir(acoustic_path)):
    if file.endswith('.mat'):
        load, fault, severity = parse_filename(file)
        try:
            mat = loadmat(os.path.join(acoustic_path, file))
            key = [k for k in mat.keys() if not k.startswith('__')][0]
            data = mat[key]
            if isinstance(data, np.ndarray):
                combined = data[:, -1] if data.ndim == 2 else data.flatten()  # 'values' column
                feats = extract_features(combined)
                feats.update({'File': file, 'Load': load, 'FaultType': fault, 'Severity': severity, 'Type': 'Acoustic'})
                acoustic_records.append(feats)
        except Exception as e:
            print(f"Error reading {file}: {e}")

acoustic_df = pd.DataFrame(acoustic_records)
print("Acoustic data loaded:", acoustic_df.shape)


100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 26.09it/s]

Error reading 0Nm_BPFI_03.mat: Cannot cast array data from dtype([('x_values', 'O'), ('y_values', 'O'), ('function_record', 'O')]) to dtype('float64') according to the rule 'unsafe'
Error reading 0Nm_BPFI_10.mat: Cannot cast array data from dtype([('x_values', 'O'), ('y_values', 'O'), ('function_record', 'O')]) to dtype('float64') according to the rule 'unsafe'
Error reading 0Nm_BPFO_03.mat: Cannot cast array data from dtype([('x_values', 'O'), ('y_values', 'O'), ('function_record', 'O')]) to dtype('float64') according to the rule 'unsafe'
Error reading 0Nm_BPFO_10.mat: Cannot cast array data from dtype([('x_values', 'O'), ('y_values', 'O'), ('function_record', 'O')]) to dtype('float64') according to the rule 'unsafe'
Error reading 0Nm_Normal.mat: Cannot cast array data from dtype([('x_values', 'O'), ('y_values', 'O'), ('function_record', 'O')]) to dtype('float64') according to the rule 'unsafe'
Acoustic data loaded: (0, 0)


In [28]:
current_temp_records = []

for file in tqdm(os.listdir(current_temp_path)):
    if file.endswith('.tdms'):
        load, fault, severity = parse_filename(file)
        try:
            tdms_file = TdmsFile.read(os.path.join(current_temp_path, file))
            df = tdms_file.as_dataframe()
            numeric_df = df.select_dtypes(include=[np.number])

            feats = {}
            for col in numeric_df.columns:
                feats[col + '_mean'] = numeric_df[col].mean()
                feats[col + '_std'] = numeric_df[col].std()
                feats[col + '_rms'] = np.sqrt(np.mean(numeric_df[col]**2))
            
            feats.update({'File': file, 'Load': load, 'FaultType': fault, 'Severity': severity, 'Type': 'Current_Temp'})
            current_temp_records.append(feats)
        except Exception as e:
            print(f"Error reading {file}: {e}")

current_temp_df = pd.DataFrame(current_temp_records)
print("Current & Temperature data loaded:", current_temp_df.shape)


100%|██████████████████████████████████████████████████████████████████████████████████| 45/45 [00:58<00:00,  1.30s/it]

Current & Temperature data loaded: (45, 20)


In [29]:
combined_df = pd.concat([vibration_df, acoustic_df, current_temp_df], axis=0, ignore_index=True)
print("Combined dataset shape:", combined_df.shape)


Combined dataset shape: (45, 20)


In [30]:
output_path = r"C:/Users/MSI/digital Twin stimulator/VATM Dataset/VATM_features.csv"
combined_df.to_csv(output_path, index=False)
print(f"✅ Features saved successfully at: {output_path}")


✅ Features saved successfully at: C:/Users/MSI/digital Twin stimulator/VATM Dataset/VATM_features.csv


In [1]:
import pandas as pd

df = pd.read_csv("VATM_features.csv")
print(df.columns)



Index(['/'Log'/'cDAQ9185-1F486B5Mod1/ai0'_mean',
       '/'Log'/'cDAQ9185-1F486B5Mod1/ai0'_std',
       '/'Log'/'cDAQ9185-1F486B5Mod1/ai0'_rms',
       '/'Log'/'cDAQ9185-1F486B5Mod1/ai1'_mean',
       '/'Log'/'cDAQ9185-1F486B5Mod1/ai1'_std',
       '/'Log'/'cDAQ9185-1F486B5Mod1/ai1'_rms',
       '/'Log'/'cDAQ9185-1F486B5Mod2/ai0'_mean',
       '/'Log'/'cDAQ9185-1F486B5Mod2/ai0'_std',
       '/'Log'/'cDAQ9185-1F486B5Mod2/ai0'_rms',
       '/'Log'/'cDAQ9185-1F486B5Mod2/ai2'_mean',
       '/'Log'/'cDAQ9185-1F486B5Mod2/ai2'_std',
       '/'Log'/'cDAQ9185-1F486B5Mod2/ai2'_rms',
       '/'Log'/'cDAQ9185-1F486B5Mod2/ai3'_mean',
       '/'Log'/'cDAQ9185-1F486B5Mod2/ai3'_std',
       '/'Log'/'cDAQ9185-1F486B5Mod2/ai3'_rms', 'File', 'Load', 'FaultType',
       'Severity', 'Type'],
      dtype='object')
